In [ ]:
from utils.animate_ball3 import BreakoutGame
%matplotlib tk
game = BreakoutGame(framesize=(64,64), xy=[[2,2],[5,2],[5,2],[5,2]], color=255, direction=[[1,1],[1,1],[1,2],[1,3]],nr_sim_steps=10,obstacle_color=200,destructible_obstacles=True,boundary_color=50,player_color=255,nr_balls=4)

for i in range(20):
    game.gs.add_obstacles([[i+10,29]])
game.main_game_loop()

In [ ]:
import gymnasium as gym
import numpy as np
from utils.dqn_agent import DQN as DQNAgent
from utils.animate_ball3 import BreakoutGame
import random
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("runs/"+str(datetime.now().strftime("%Y%m%d-%H%M%S")))
# We create our gym environment 
#env = BreakoutGame(framesize=(64,64), xy=[[5,2]], color=255, direction=[[1,1]],nr_sim_steps=10,obstacle_color=200,destructible_obstacles=False,boundary_color=50,player_color=255,nr_balls=1)
# We get the shape of a state and the actions space size
state_size = 1*2*2+1
action_size = 3#left,right,none
# Number of episodes to run
n_episodes = 15000

# Max iterations per epiode
max_iteration_ep = 500
# We define our agent
agent = DQNAgent(state_size, action_size)
agent.lr=1e-6
agent.gamma=0.99
agent.exploration_proba=1
agent.exploration_proba_decay=0.001
agent.batch_size=32
agent.verbose=True
batch_size=agent.batch_size
total_steps = 0

# We iterate over episodes
for e in range(n_episodes):
    # We initialize the first state and reshape it to fit 
    #  with the input layer of the DNN
    dirx = random.random()*8-4
    diry = random.random()*8-4#
    dirx=random.choice([-2,-1,0,1,2])
    diry=random.choice([-2,-1,1,2])
    xy=[random.randint(2,28),random.randint(10,28)]

    env = BreakoutGame(framesize=(32,32), xy=[xy], color=255, direction=[[dirx,diry]],nr_sim_steps=10,obstacle_color=200,destructible_obstacles=False,boundary_color=50,player_color=255,nr_balls=1)
    current_state = env.gs.get_state()
    current_state = np.array([current_state])


    for step in range(max_iteration_ep):
        
        total_steps = total_steps + 1
        # the agent computes the action to perform
        action = agent.compute_action(current_state)
        if action==0:
            action=[0,0]
        if action==1:
            action=[2,0]
        if action==2:
            action=[-2,0]
        # the envrionment runs the action and returns
        # the next state, a reward and whether the agent is done

        ##next_state, reward, done, _,_ = env.step(action)
        next_state, reward, done = env.rl_refresh_frame(action)
        #print(current_state,step,reward)
        next_state = np.array([next_state])
        #print(next_state)
        
        # We sotre each experience in the memory buffer
        #print("current_state",current_state,"action",action,"reward",reward,"next_state",next_state,"done",done)
        agent.store_episode(current_state, action, reward, next_state, done)
        
        # if the episode is ended, we leave the loop after
        # updating the exploration probability
        if done:
            agent.update_exploration_probability()
            break
        current_state = next_state
    # if the have at least batch_size experiences in the memory buffer
    # than we tain our model
    if total_steps >= batch_size:
        loss=agent.train()
        try:
            writer.add_scalar("Loss/train", loss, e)
        except:
            pass


In [ ]:
import matplotlib.pyplot as plt
%matplotlib tk
plt.ion()
fig = plt.figure(1)
env = BreakoutGame(framesize=(32,32), xy=[[20,20]], color=255, direction=[[3,1]],nr_sim_steps=10,obstacle_color=200,destructible_obstacles=False,boundary_color=50,player_color=255,nr_balls=1)


while env.has_ended==False:
    current_state = env.gs.get_state()
    current_state = np.array([current_state])
    action = agent.compute_action(current_state)
    if action==0:
        action=[0,0]
    if action==1:
        action=[2,0]
    if action==2:
        action=[-2,0]
    env.gs.player_direction=action
    frame=env.refresh_frame()
            #frameT = frame.transpose((1,0))
            #plt.clf()
    plt.imshow(np.flip(frame.T,0))
    plt.pause(0.03)
    plt.clf()